In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import os
import PIL

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import cv2
import csv
import PIL

Dataset_Path = os.path.join(os.getcwd(),'inaturalist_12K')
# Dataset_Path = '/content/drive/MyDrive/inaturalist_12K'
train_path = os.path.join(Dataset_Path , 'train')
test_path = os.path.join(Dataset_Path , 'val')
# test_path1 = test_path
import pathlib
Dataset_Path = pathlib.Path(Dataset_Path)
train_path = pathlib.Path(train_path)
test_path = pathlib.Path(test_path)

In [3]:
# import tensorflow as tf
# tf.test.gpu_device_name()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0],
                                                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5500)])
  except RuntimeError as e:
    print(e)

In [4]:
batch_size = 64
IMAGE_SIZE = 256

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True, 
    seed=1234, 
    validation_split=0.1,
    subset="training",
    interpolation='bilinear'
)

Found 9999 files belonging to 10 classes.
Using 9000 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True, 
    seed=1234, 
    validation_split=0.1,
    subset="validation",
    interpolation='bilinear'
)

Found 9999 files belonging to 10 classes.
Using 999 files for validation.


In [7]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path, 
    class_names=None, 
    color_mode='rgb', 
    batch_size=batch_size, 
    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
    shuffle=True,
    seed=1234,
    interpolation='bilinear'
)

Found 2000 files belonging to 10 classes.


In [8]:
class_names = train_ds.class_names
print(class_names)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Dropout , BatchNormalization, experimental

In [11]:
data_augmentation = Sequential(
  [
    experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(IMAGE_SIZE, 
                                                              IMAGE_SIZE,
                                                              3)),
    experimental.preprocessing.RandomRotation(0.2),
    experimental.preprocessing.RandomZoom(0.2),
  ]
)

# Pretrained Model

In [12]:
def save_pretrained_model(model_name, epochs=10):
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(10,activation='softmax')
    inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = data_augmentation(inputs)
    if model_name == 'InceptionV3':
        base_model = tf.keras.applications.InceptionV3(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3), include_top=False, weights='imagenet')
        x = tf.keras.applications.imagenet_utils.preprocess_input(x,mode='tf')
    elif model_name == 'InceptionResNetV2':
        base_model = tf.keras.applications.InceptionResNetV2(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
        x = tf.keras.applications.imagenet_utils.preprocess_input(x,mode='tf')
    elif model_name == 'ResNet50':
        base_model = tf.keras.applications.ResNet50(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
        x = tf.keras.applications.imagenet_utils.preprocess_input(x,mode='caffe')
    elif model_name == 'Xception':
        base_model = tf.keras.applications.Xception(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
        x = tf.keras.applications.imagenet_utils.preprocess_input(x,mode='tf')
    else:
        base_model = None
        preprocess_input = None
    base_model.trainable = False
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
    initial_epochs = epochs
    history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)
    model.save('PartB_models' + os.sep + 'base_models' + os.sep + model_name)
    model.save('PartB_models' + os.sep + 'models' + os.sep + model_name)
#     return model,history,base_model 

In [13]:
# for model_name in ['InceptionV3','InceptionResNetV2','ResNet50','Xception']:
#     save_pretrained_model(model_name)

In [14]:
def get_pretrained_model(model_name, epochs=10):
    base_model = tf.keras.models.load_model('PartB_models' + os.sep + 'base_models' + os.sep + model_name)
    model = tf.keras.models.load_model('PartB_models' + os.sep + 'models' + os.sep + model_name)
    return base_model, model

In [15]:
# get_pretrain_conv_model('InceptionV3',10)

# Fine tuning

In [16]:
from wandb.keras import WandbCallback

In [17]:
def finetune(sweep_id):
    config_defaults = {
        'epochs' : 10,
        'model_name': 'InceptionV3',
        'strategy': '1'
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    model,base_model = get_pretrained_model(config.model_name,config.epochs)
    print("Number of layers in base Model: ", len(base_model.layers))
    n = len(base_model.layers)
    l = base_model.layers
    fine_tune_at = n
    base_model.trainable = True
    if config.strategy == 0:
        fine_tune_at = n
    if config.strategy == 1:
        fine_tune_at = n-1
    if config.strategy == 2:
        fine_tune_at = n-20
    if config.strategy == 3:
        fine_tune_at = n-50
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable =  False   
    print(len(model.trainable_variables))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(lr=0.00001),
              metrics=['accuracy'])
    
    fine_tune_epochs = config.epochs
    total_epochs = epochs + fine_tune_epochs
    history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds,
                            callbacks=[WandbCallback()])
    model.save('models'+os.sep+str(sweep_id)+os.sep+wandb.run.name)

        

# Wandb Sweep

In [18]:
import wandb
wandb.login(key="866040d7d81f67025d43e7d50ecd83d54b6cf977", relogin=False)

wandb: Currently logged in as: mooizz (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Jaitesh/.netrc


True

In [19]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model_name': {  
            'values': ['InceptionV3','InceptionResNetV2','ResNet50','Xception']
        },
        'strategy':{
            'values': [1,2,3]
        }
    }
}

In [20]:
sweep_id = wandb.sweep(sweep_config, entity="mooizz",project="conv_inaturalist")

Create sweep with ID: kulg4dit
Sweep URL: https://wandb.ai/mooizz/conv_inaturalist/sweeps/kulg4dit


In [21]:
wandb.agent(sweep_id, finetune(sweep_id))

Number of layers in base Model:  8
190


NameError: name 'epochs' is not defined